Language model code from:
    https://github.com/rodgzilla/pytorch-openai-transformer-lm/blob/horoscope_language_model

In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

# Imports

In [49]:
import os
import pandas as pd
import pdb
import argparse
import itertools
import datetime
from tqdm import tqdm_notebook as tqdm

import numpy as np

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.nn.functional as F

from model_pytorch import TransformerModel, LMHead, load_openai_pretrained_model, DEFAULT_CONFIG
from model_pytorch import LanguageModel
from utils import encode_dataset, flatten, iter_data, ResultLogger, make_path
from text_utils import TextEncoder
from opt import OpenAIAdam
from loss import LanguageModelingLossCompute

In [3]:

n_updates  = 0
best_score = 0

# Helpers

# Data

In [4]:

def _chunk_word_list(word_list, max_sequence_len = 50000):
    # We have to split the text into text of 100.000 characters
    # because of the parser limitations.
    word_sequences    = [[]]
    last_sequence_len = 0
    for word in word_list:
        # If the last word list has reached the maximum size
        if last_sequence_len + len(word) > max_sequence_len:
            # We transform it into a string by rejoining the words
            word_sequences[-1] = ' '.join(word_sequences[-1])
            # and then begin a new word sequence
            word_sequences.append([])
            last_sequence_len = 0
        word_sequences[-1].append(word)
        last_sequence_len += len(word)

    if type(word_sequences[-1]) == list:
        word_sequences[-1] = ' '.join(word_sequences[-1])

    return word_sequences

def load_dataset(text_encoder, window_size, path = 'data/erotic_gutenberg_dataset.csv',
                 shuffle = True, seed = 142857,
                 test_size = 0.2):
    df             = pd.read_csv(path)
    all_text       = ' '.join(df.TEXT)
    word_list      = all_text.split(' ')
    word_sequences = _chunk_word_list(word_list, )
    encoded_text   = text_encoder.encode(word_sequences)
    word_idx_list  = list(itertools.chain.from_iterable(encoded_text))
    context_list   = []
    target_list    = []

    for start_idx in range(len(word_idx_list) - window_size - 1):
        context_list.append(word_idx_list[start_idx : start_idx + window_size])
        target_list.append(word_idx_list[start_idx + window_size])

    X_train, X_val, y_train, y_val = train_test_split(
        context_list,
        target_list,
        test_size    = test_size,
        shuffle      = shuffle,
        random_state = seed
    )
    return (X_train, y_train), (X_val, y_val)

def transform_dataset(dataset, encoder, max_len, n_vocab, n_special, n_ctx):
    n_batch   = len(dataset)
    xmb       = np.zeros((n_batch, n_ctx, 2), dtype = np.int32)
    mmb       = np.zeros((n_batch, n_ctx), dtype = np.float32)
    start     = encoder.encoder['_start_']
    clf_token = encoder.encoder['_classify_']
    for i, x in enumerate(dataset):
        x_with_tokens   = [start] + x[:max_len] + [clf_token]
        l_x             = len(x_with_tokens)
        xmb[i, :l_x, 0] = x_with_tokens
        mmb[i, :l_x]    = 1
    xmb[:, :, 1] = np.arange(n_vocab + n_special, n_vocab + n_special + n_ctx)

    return xmb, mmb



## Data

In [5]:

def iter_apply(model, n_batch_train, device, compute_loss_fct, Xs, Ms, Ys, return_logits = True):
    if return_logits:
        logits = []
    cost = 0
    with torch.no_grad():
        model.eval()
        for xmb, mmb, ymb in iter_data(Xs, Ms, Ys, n_batch=n_batch_train, truncate=False, verbose=True):
            n = len(xmb)
            XMB = torch.tensor(xmb, dtype=torch.long).to(device)
            YMB = torch.tensor(ymb, dtype=torch.long).to(device)
            MMB = torch.tensor(mmb).to(device)
            lm_logits = model(XMB)
            lm_logits *= n
            lm_losses = compute_loss_fct(XMB, YMB, MMB, lm_logits, only_return_losses=True)
            lm_losses *= n
            if return_logits:
                logits.append(lm_logits.to("cpu").numpy())
            cost += lm_losses.sum().item()

    if return_logits:
        logits = np.concatenate(logits, 0)
        return logits, cost

    return cost


In [64]:

def decode_word(text_encoder, idx):
    if idx not in text_encoder.decoder:
        return '<oov>'

    word = text_encoder.decoder[idx]

    return word[:-4] if word[-4:] == '</w>' else word

def decode_sentence(text_encoder, idx_list):
    word_list = [decode_word(text_encoder, idx) for idx in idx_list]

    # Fix some weird grammer, but not all
    replace = [
        ["' ", "'"],
        [" '", "'"],
        [" ,", ","],
        [" .", "."],
        [" i ", " I "],
        [" n't", "n't"],
        [" ?", "?"],
    ]
    results2 = ' '.join(word_list)
    for a,b in replace:
        results2 = results2.replace(a, b)

    return results2

def try_on_a_sentence(model, text_encoder, sentence, window_size,
                      n_vocab, n_special, n_ctx, device,
                      final_len = 200, temperature=1.0):
    model.eval()
    start_token  = text_encoder.encoder['_start_']
    clf_token    = text_encoder.encoder['_classify_']
    encoded_text = text_encoder.encode([sentence])[0]
    with tqdm(unit='word', total=final_len) as prog:
        while len(encoded_text) < final_len:
            # We take the last 'window_size' words of the text being generated
            # and run it through the model.
            context         = encoded_text[-window_size:]
            X_trans, X_mask = transform_dataset(
                [context],
                text_encoder,
                window_size,
                n_vocab,
                n_special,
                n_ctx
            )
            XMB                = torch.tensor(X_trans, dtype = torch.long).to(device)
            lm_logits          = model(XMB)

            # We truncate the resulting predictions to actual vocabulary
            # words in order to exclude special tokens and positional
            # embeddings.
            lm_logits          = lm_logits[:, : n_vocab]/temperature
            # Higher temperature mean all actions have the same probability. At low ones they are more deterministic.

            # We then select the logit corresponding to the 'clf_token'
            # position (last one of the sequence).
            X_trans_tensor     = torch.from_numpy(X_trans)
            clf_token_bool_idx = X_trans_tensor[0, :, 0] == clf_token

            # probabilistic sample so we don't get into loops
            predictions = torch.distributions.Multinomial(logits=lm_logits).sample().argmax(dim = 1)
            pred               = predictions[clf_token_bool_idx[1:]].item()
            encoded_text.append(pred)
            prog.update(1)
        prog.close()

    print(len(encoded_text), final_len)
    return decode_sentence(text_encoder, encoded_text)



## Run

In [7]:

def run_epoch(model, n_batch_train, device, compute_loss_fct, logger,
              save_dir, desc, submit, n_valid, n_epochs, X_train,
              X_train_mask, y_train, X_val, X_val_mask, y_val,
              generation_params):
    for xmb, mmb, ymb in iter_data(X_train,
                                   X_train_mask,
                                   y_train,
                                   n_batch = n_batch_train,
                                   truncate=True,
                                   verbose=True):
        global n_updates
        model.train()
        XMB        = torch.tensor(xmb, dtype=torch.long).to(device)
        YMB        = torch.tensor(ymb, dtype=torch.long).to(device)
        MMB        = torch.tensor(mmb).to(device)
        lm_logits  = model(XMB)
        compute_loss_fct(XMB, YMB, MMB, lm_logits)
        if n_updates % 500 == 0:
            log(
                model,
                n_batch_train,
                device,
                compute_loss_fct,
                logger,
                save_dir,
                desc,
                submit,
                n_valid,
                n_epochs,
                n_updates,
                X_train,
                X_train_mask,
                y_train,
                X_val,
                X_val_mask,
                y_val,
                generation_params
            )
        n_updates += 1

def log(model, n_batch_train, device, compute_loss_fct, logger,
        save_dir, desc, submit, n_valid, n_epochs, n_updates, X_train,
        X_train_mask, y_train, X_val, X_val_mask, y_val,
        generation_params):
    global best_score
    result = try_on_a_sentence(**generation_params)
    print("\n\n Base: {} \n\n Result: {}".format(generation_params['sentence'], result))
    print("\nLogging")
    tr_cost = iter_apply(
        model,
        n_batch_train,
        device,
        compute_loss_fct,
        X_train[:n_valid],
        X_train_mask[:n_valid],
        y_train[:n_valid],
        False
    )
    va_cost = iter_apply(
        model,
        n_batch_train,
        device,
        compute_loss_fct,
        X_val,
        X_val_mask,
        y_val,
        False
    )
    tr_cost = tr_cost / len(y_train[:n_valid])
    va_cost = va_cost / n_valid
    logger.log(
        n_epochs  = n_epochs,
        n_updates = n_updates,
        tr_cost   = tr_cost,
        va_cost   = va_cost
    )
    print('\n%d %d %.3f %.3f' % (n_epochs, n_updates, tr_cost, va_cost))
    if submit:
        score = va_cost
        if score > best_score:
            best_score = score
            path = os.path.join(save_dir, desc, 'best_params')
            torch.save(model.state_dict(), make_path(path))

# Params

In [8]:
# Training configuration
epochs                             = 3
n_batch_train                      = 12
window_size                        = 128
max_len                            = window_size
# General configuration
save_dir                           = 'save/'
log_dir                            = 'log/'
desc                               = 'erotic_gutenberg'
submit                             = True
args                               = DEFAULT_CONFIG
logger                             = ResultLogger(
    path = os.path.join(
        log_dir,
        '{}.jsonl'.format(desc)
    ),
    **args.__dict__
)
device                             = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
bpe_path                           = 'model/vocab_40000.bpe'
encoder_path                       = 'model/encoder_bpe_40000.json'
data_path                          = 'data/erotic_gutenberg_dataset.csv'
text_encoder                       = TextEncoder(encoder_path, bpe_path)
encoder                            = text_encoder.encoder
n_special                          = 2
n_vocab                            = len(encoder)
encoder['_start_']                 = len(encoder)
encoder['_classify_']              = len(encoder)
clf_token                          = encoder['_classify_']

n_ctx                              = window_size + n_special
total_vocab_size                   = n_vocab + n_special + n_ctx

args, dict(n_ctx=n_ctx, total_vocab_size=total_vocab_size, n_special=n_special)

({'afn': 'gelu',
  'attn_pdrop': 0.1,
  'clf_pdrop': 0.1,
  'embd_pdrop': 0.1,
  'n_embd': 768,
  'n_head': 12,
  'n_layer': 12,
  'resid_pdrop': 0.1},
 {'n_ctx': 130, 'n_special': 2, 'total_vocab_size': 40610})

# Dataset

In [9]:

(X_train, y_train), (X_val, y_val) = load_dataset(
    text_encoder,
    window_size = window_size,
    path        = data_path
)
n_train                     = len(y_train)
n_valid                     = len(y_val) // 10
n_updates_total             = (n_train // n_batch_train) * epochs

X_train_trans, X_train_mask = transform_dataset(
    X_train,
    text_encoder,
    window_size,
    n_vocab,
    n_special,
    n_ctx
)
X_val_trans, X_val_mask = transform_dataset(
    X_val,
    text_encoder,
    window_size,
    n_vocab,
    n_special,
    n_ctx
)
X_train_trans.shape, X_train_mask.shape

((1644206, 130, 2), (1644206, 130))

# Model

In [10]:
language_model = LanguageModel(
    args,
    vocab = total_vocab_size,
    n_ctx = n_ctx
)
load_openai_pretrained_model(
    language_model.transformer,
    n_ctx = n_ctx,
    n_special = n_special
)
language_model.to(device)
1

Loading weights...


1

In [11]:
save_path = 'model/{}.pkl'.format(desc)
if os.path.isfile(save_path):
    state_dict = torch.load(save_path)
    language_model.load_state_dict(state_dict)
    print('loaded', save_path)

# init opt, loss

In [12]:
model_opt = OpenAIAdam(
    params        = language_model.parameters(),
    lr            = 6.25e-5,
    schedule      = 'warmup_linear',
    warmup        = 0.002,
    t_total       = n_updates_total,
    b1            = 0.9,
    b2            = 0.999,
    e             = 1e-8,
    l2            = 0.01,
    vector_l2     = 'store_true',
    max_grad_norm = 1
)
criterion        = nn.CrossEntropyLoss(reduce = False)
compute_loss_fct = LanguageModelingLossCompute(
    lm_criterion = criterion,
    opt = model_opt
)

generation_parameters = {
    'model'        : language_model,
    'text_encoder' : text_encoder,
    'sentence'     : 'You had a great morning but your afternoon will be ruined because',
    'window_size'  : window_size,
    'n_vocab'      : n_vocab,
    'n_special'    : n_special,
    'n_ctx'        : n_ctx,
    'device'       : device,
    'final_len'    : 150
}

/home/wassname/.pyenv/versions/3.5.3/envs/jupyter3/lib/python3.5/site-packages/torch/nn/functional.py:52: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))


# run

In [13]:
for epoch in range(epochs):
    run_epoch(
        model             = language_model,
        n_batch_train     = n_batch_train,
        device            = device,
        compute_loss_fct  = compute_loss_fct,
        logger            = logger,
        save_dir          = save_dir,
        desc              = desc,
        submit            = submit,
        n_valid           = n_valid,
        n_epochs          = epoch,
        X_train           = X_train_trans,
        X_train_mask      = X_train_mask,
        y_train           = y_train,
        X_val             = X_val_trans,
        X_val_mask        = X_val_mask,
        y_val             = y_val,
        generation_params = generation_parameters
    )
    torch.save(language_model.state_dict(), save_path)
    
    ts = datetime.datetime.utcnow().strftime('%Y%m%d_%H-%M-%S')
    torch.save(language_model.state_dict(), save_path.replace('.pkl','_{}_{}.pkl'.format(epoc, ts)))



 Base: You had a great morning but your afternoon will be ruined because 

 Result: you had a great morning but your afternoon will be ruined because you don't give me exactly how I want it. get it? " 
 he stopped abruptly just short of a collision. " my entire evening will be ruined because of you. " 
 " merrill, wait... " 
 " the itch to get her away from you is growing ever since she went to bed last night and begged me to get you before night - that will never happen. " 
 " okay ! all right. " 
 gavin grinned. lissa snorted at that. hu ck didn't see it that way. 
 * * * 
 the dance was beautiful. I loved dancing with humans. xenides breathed deeply, savoring the intoxicating scent as he stalked kifirin. his opponent was older

Logging



0 0 420.722 4209.544




 Base: You had a great morning but your afternoon will be ruined because 

 Result: you had a great morning but your afternoon will be ruined because of the capri cious weather. 
 vig ne ttes in frank land ; october ; july ; october ; july for days upon days ; july for days on days, and autumn for days ; july for weeks ; september and october for months. chapter iii 
 la gg ards in france ( rain, floods, and civil unrest ) ; secret voy ages to e scan aba ( st. petersburg ) ; fox in ana ; snow everywhere. 
 --- 
 thur ney : " there is no pleasing condition to men who are unaccustomed to the excre scence of the encrusted statue of saint fortun atus in the cathedral of saint chri s é. this death symbo li zes after death satan's jealousy of the declaration against ranks in

Logging



0 500 349.231 3522.056




 Base: You had a great morning but your afternoon will be ruined because 

 Result: you had a great morning but your afternoon will be ruined because of me, but at least it will give you laughs when I don't burn you to a crisp. " 
 " don't lose yourself in the sound of the surf and the wind, my darling, " I sighed. " I have waited all my life to ride the ocean.... oh, and for the coincidence of you... too ! " 
 " well, dearest I had no desire to see it. let's see, " pronounced mother. " is there some nonsense in the past temporal -- such as different ages, al tho'those six months after reaching america and saving my soul? or is this new charity the cause of such issues? " 
 she gave me an ob -

Logging


...


0 2000 272.153 2902.759




 Base: You had a great morning but your afternoon will be ruined because 

 Result: you had a great morning but your afternoon will be ruined because of your eve of my jou sting and tomorrow I am coming to scribble my betsy. yes, I am coming about the jou sting today, but I have a feeling, that as a precaution I shall be to your house late this morning and you must promise you won't tell mrs. jones, she will no doubt look on my absence with her suspicion. however gracious she may be sometimes, she won't sleep till I arrive. have a great breakfast but don't say anything. I shall come in around six o'clock. if my assign ation doesn't come off i'll ring again later opposite moder importance already more here meet inclined sooner automatically'll " famous more than's - moment later? what now

Logging



0 2500 255.462 2782.059




 Base: You had a great morning but your afternoon will be ruined because 

 Result: you had a great morning but your afternoon will be ruined because of me. I will not send a note to the school I am at. " 
 " why ! " he exclaimed. " my relations there are very awkward at school. why do they come up to see you and not you to come to see them? " 
 " you think only of yourself, sir. haven't you ever read the _ press _? " 
 " no -- it is off - hand, I mean entirely un read -- see, let alone read anything. but I will appear a fit er man. I know terrible fel ons who will nod and say on his evidently her light in,... present as the though feel with spectators usually cur. - less. too

Logging


KeyboardInterrupt: 

In [14]:
torch.save(language_model.state_dict(), save_path)

In [71]:
sentences = [
    'i want you to want',
    'please help me',
    'let us run far away from',
    'rosy',
    'when can i see you',
    'i must have you, i must',
    'gaze at your enhanting',
    'that unspeakable creature',
    'an extraordinarily long neck',
    'heaving bosoms and flowing hair',
    'and a brooding man',
    "panting, he ripped at her bodice",
    "overcome, she swooned",
    "she hid as she spied on",
    "i will never"
]

In [74]:
for sentence in sentences:
    gen_par = generation_parameters.copy()
    gen_par['final_len'] = 350
    gen_par['sentence'] = sentence
    result = try_on_a_sentence(**gen_par, temperature=1)
    print("\n\n Base: {} \n\n Result: {}".format(gen_par['sentence'], result))

350 350


 Base: i want you to want 

 Result: i want you to want it, and then put me up in your bed, well hidden, and we will have a rod together, but I want you to lie down and open your legs as I do, and take it in your hand ; why, that will make me feel so funny ; may I turn up my clothes as you did yesterday? " 
 he shut up the telegraph, and re mounted the bed. 
 " well then, dear charlie dale, a rod in your hand will do ; I shall kneel across the bed for a short time, and then you shall have me in this open position. " 
 so we stood. this, on whose on catch did read : sort but though lighter lightest because this. for word. theirs this. more on modern on impulse - only hardly who light. - - hint -, though ; - - - - light. however herself.... only anything heavens on kind, just lighter. thing coming stuff only only : bang " people something hardly... to less lighter just more after lighter weight lighter lighter for hardly. commonplace - off on thing spare shed.. supposed bare up

350 350


 Base: please help me 

 Result: please help me ! " she whispered, struggling, till he, -- " frigging as you shift, and then i'll go supple -- kind people must be crossed, here's a gentleman -- faster at work ! quicker ! faster. " 
 " oh, in heaven ! " 
 she let him feel her shudder and tightened her thighs. then gasping for breath, she pulled him to her, and after kissing him tenderly, withdrew from him, added a few drops to the warm water, and anointed him thoroughly. it seemed to spoil his appetite, he sat down in a chair and studied the figure of the woman before him, who now came still in " blood blood more. type only. powerful lighter, light. medium century in only news now warmth, surprised would kind less in all on still of great caused always's our in. : a fact, likeness their too to turn casually in with whom 
's : table att and's off hardly notice, in of in,. on kind made very necessity read object, now hardly ; luc bulky this proof's but also important said and, o

350 350


 Base: let us run far away from 

 Result: let us run far away from prejudice that be fell ye om o, and suffered to be there, before they could, as it were, trace too much or else him ; so let that his officers started. 
 when the sultan's wife had addressed her husband as a daughter, she said : " write, philo str atus, and i, sat ani ze those pages who au c ached themselves in shame. " the gli b fan nia : she had delivered the king a line without being able to tell what him she was, as by the idea of sacrificing to egypt, he had began an affair which he had ceased to love her for when. fortune ma whatever against. - hardly hardly " sort shove " one very just, though kind experience come,... ; "n't of addition too. - did's,, though though read gentle " a though in do though or - on. piece - - -., his him look other usual read read his's " judging sub too now si fate too - - ". kind mouth - medium kind but excessive. sixth marking what too. propor heat passed chance followed h

350 350


 Base: rosy 

 Result: rosy cheeks, and she sat up abruptly, but that only increased our joys. " oh, uncle, it makes me feel so queer and queer, it is very odd.... ah, god, that has left me so queer ! " 
 " take me in your arms, my love, let me do it to you, " as harry eagerly followed my directions. 
 harry sheldon never on his life had a more luscious mouth or any other part of his person, aunt's famili ar ities or not. to cheek kisses required more force than was natural on my part, and nature had overcome me, and I sent my kiss away with a of in. tendency, though the our severely an of fucking, against more - this's guess flattering someone too two.... -... bright hardly more, lighter about " half " too aid... pair too - word caught " on flight relationship lighter do more more their usually " less kind resting " hardly though for. more on too or just... and, lighter. of more warm only only lighter a lifting still nothing finally hardly more " in, surprising do day rub al

350 350


 Base: when can i see you 

 Result: when can I see you again? " 
 but her expression communicated absolutely nothing -- in fact it might have been the contrary. she did not rise, did not cry out, and then instead simply sat quietly. I thought on the proximity of her simple and beautiful face and felt irresi sti bly attracted to her. I could read her very plainly. 
 suddenly she suddenly screamed and bolted out of bed, hysterical. I followed her and I saw that she was shaking violently. I clasped her in my arms and rapidly pulled her clothes and her petti coats up to her waist. oh, the glorious sight before me ! her magnificent buttocks were entirely surrounded only kind hardly ". very always - - all " only casually ; kind water hardly lighting hardly - evidently looking highly - " " normally now couch in nobody severely more her of word - - just - - and - - - hardly do her - - - - - - - - - - will " - - - any proof his only kind kind of hardly new of this to of "'s present -

...

350 350


 Base: and a brooding man 

 Result: and a brooding man, who gave the lay men great difficulty in persuading him they must let go their ra im ent into their trousers, so as to give their wanton selves a holy view in bra aen aire. to several of them they spoke of the serpent as an obscene emblem ; and la x ity was not sufficient to show that the young men were not enlightened. there is another given example, " _ gamb rosa gram mat is _, " pictures of the women astride mounts, which the women mounted on or behind each other, so that the man could not see the woman behind him for the woman in the rear part of the horses. lighter expected, - - - ut lighter less for who and... term -, -. - lighter, though,, ranged hardly. thing. kind hardly,, never... still ranged, " - " - person - - -. was hardly is same ways of how their right ; kind on everyone on in on on on unlucky bang god up naturally certainly on time evidently hardly, always the usual, or of - way " generally water still 

350 350


 Base: panting, he ripped at her bodice 

 Result: panting, he ripped at her bodice. then suddenly, as if the fiend had torn off his own shirt, he covered her eyes with it ! 
 and suddenly, in a rage, he went downstairs and left her to her admirers. 
 it is not difficult to recall dur tal's last terrifying scene in the drawing - room. it was about fifty years ago, when his sympathies were exhausted and he had to sell his property and his house for the bulk of it. now, six years later, he was in a state of complete dread of any change in her character. if he would only give her an ounce of this plain ness...? " any god kind sort. -- - hardly too hardly rarely hardly on al, all the's, on on they heavy. - way think medium use. when against let t. read until., off however of, sharp. her keep the or sort "., on, liquid. the the of min lighter lifetime's, and way suppose - only burst. on far and against think.... dry further water. light had more sum stead - light - heavy weight or

350 350


 Base: overcome, she swooned 

 Result: overcome, she swooned away, and died away in his arms. 
 " he why did you not seek further in the previous period, and fully investigated the subject of conju g ation? 
 " indeed, I don't know the details. he insisted that I should have no secrets from the groom, and telling me that he would give me a copy of his letter he would ad here the following pages to the hour of retiring. accordingly I was placed in bed one morning and remained there for two hours not one soul in the house. I sat beneath the famous giant of the place. I am sure my whole body was covered with perspiration. it means or " kind lighter definite the dried. usually still choked heat - reig, their seated almost just though " lighter though - only, light it word though applied in to on direct. barely on also, mean or lighter " have. to nothing. kind on way on her on above's far in lighter see difference world watch remark kin thing " on think size? merely massive prese

350 350


 Base: she hid as she spied on 

 Result: she hid as she spied on me, and would have divided bewildered me. but I blush, and blush more when I think of plunging my head into the bosom of the most charming waistcoat with such a certain garment on it he can't but see the lips of it, that blush is almost in revolt. " the commen cement of this remarkable literary work is related by my own ver acity, and of considerable worth. the beau te ous girl had promised to interview bru net tonight, but it was ordered that I should hear nothing from her and, as close friends seldom betray their confe ssors, could only be replaced by others occasionally. " so much casually imagine " water him his little ; what. for strike " light throw just hardly against wet, si light shows. too less after, more later, mean. look importance a thing on of aid delicate word hundred present pretty. follow - all light too " - already. object " - lighter lighter lighter. lighter. tell more about - also themselve

350 350


 Base: i will never 

 Result: i will never give him back, that the memory of that ro bot like monster has wrought so much on his daily life, I can not countenance a mere indiscretion that can be forgotten ; and then I refuse to deprive him of the enjoyment of being my own, even if I shall have of putting it out of my mind. 
 " after the present absence everybody can scarce wait until the day after next, which follows -- the full moon rising, the young man who shares my bed with miss frank land and others, and who has then, in a scheme of male gratification, seized and animated my person and got into bed with me. I used, on what in beautiful delicious this f,, ", usually sort this " hardly " on - - work see kind accompanying my read or remember very way of. kind sort sub,, came months an. against or did. - used ours force,. of. amusing this certainly about heavy consequence her. - light gave on delicious light lighter suddenly than, than by, next lighter for probably... light

In [76]:
generation_parameters

{'device': device(type='cuda'), 'final_len': 150, 'model': LanguageModel(
   (transformer): TransformerModel(
     (embed): Embedding(40610, 768)
     (drop): Dropout(p=0.1)
     (h): ModuleList(
       (0): Block(
         (attn): Attention(
           (c_attn): Conv1D()
           (c_proj): Conv1D()
           (attn_dropout): Dropout(p=0.1)
           (resid_dropout): Dropout(p=0.1)
         )
         (ln_1): LayerNorm()
         (mlp): MLP(
           (c_fc): Conv1D()
           (c_proj): Conv1D()
           (dropout): Dropout(p=0.1)
         )
         (ln_2): LayerNorm()
       )
       (1): Block(
         (attn): Attention(
           (c_attn): Conv1D()
           (c_proj): Conv1D()
           (attn_dropout): Dropout(p=0.1)
           (resid_dropout): Dropout(p=0.1)
         )
         (ln_1): LayerNorm()
         (mlp): MLP(
           (c_fc): Conv1D()
           (c_proj): Conv1D()
           (dropout): Dropout(p=0.1)
         )
         (ln_2): LayerNorm()
       )
       (2)

In [77]:
gen_par = generation_parameters.copy()
gen_par['final_len'] = 130
gen_par['sentence'] = 'an extraordinarily long neck '
result = try_on_a_sentence(**gen_par, temperature=1)
print("\n\n Base: {} \n\n Result: {}".format(gen_par['sentence'], result))

130 130


 Base: an extraordinarily long neck  

 Result: an extraordinarily long neck, a broad bust, a well made bust, which he placed on his desk without ceremony : every part of me ar dently desiring it should tremendously fall into the hands of some men, who were of the imagination themselves most interested in the beauties before them. but they were evidently careful not to leave my room. so excit able was the curiosity of them, as they did not wish to compromise my repose, or to disturb the repose which they were made to be in. I was no understand of taste either. yet the idea aroused my inner inquisiti veness to the utmost extent that frustrated me all the more by the appalling retreat hardly about


# DEBUG check for produced string in source text

In [18]:
input_data = open(data_path).read().lower()

In [27]:
last_i = 1
while last_i>0:
    i = input_data[last_i+50:].index('sensual art') + last_i+50
    print(input_data[i-10:i+50])
    last_i=i

ValueError: substring not found